In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import sys
sys.path.append('/mnt/d/Forecasting/liquid_time_constant_networks/experiments_with_ltcs/')
from models.ts_model import TS_Data, TSModel
import argparse
import datetime as dt

os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Run on CPU

2023-09-10 11:54:10.477759: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-10 11:54:10.762398: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-10 11:54:10.763999: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-10 11:54:11.786527: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def load_trace():
    df = pd.read_csv("../data/traffic/Metro_Interstate_Traffic_Volume.csv")
    holiday = (df["holiday"].values == None).astype(np.float32)
    temp = df["temp"].values.astype(np.float32)
    temp -= np.mean(temp)  # normalize temp by annual mean
    rain = df["rain_1h"].values.astype(np.float32)
    snow = df["snow_1h"].values.astype(np.float32)
    clouds = df["clouds_all"].values.astype(np.float32)
    date_time = df["date_time"].values
    # 2012-10-02 13:00:00
    date_time = [dt.datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in date_time]
    weekday = np.array([d.weekday() for d in date_time]).astype(np.float32)
    noon = np.array([d.hour for d in date_time]).astype(np.float32)
    noon = np.sin(noon * np.pi / 24)

    features = np.stack([holiday, temp, rain, snow, clouds, weekday, noon], axis=-1)

    traffic_volume = df["traffic_volume"].values.astype(np.float32)
    traffic_volume -= np.mean(traffic_volume)  # normalize
    traffic_volume /= np.std(traffic_volume)  # normalize

    return features, traffic_volume

In [3]:
x, y = load_trace()

In [4]:
x.shape, y.shape

((48204, 7), (48204,))

In [5]:
model_type = 'ltc'
model_size = 32
epochs = 2
seq_len = 6
log = -1
batch_size = 16

In [6]:
input_ts_data = TS_Data(x = x, y = y, seq_len = seq_len, batch_size = batch_size)

Total number of training sequences: 12050


In [7]:
model = TSModel(model_type=model_type, model_size=model_size)
model.fit(input_ts_data, epochs=epochs, log_period=log)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
logit shape:  (?, ?, 1)


2023-09-10 11:54:28.785462: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-10 11:54:28.807068: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled


Epochs 000, train loss: 0.27, train mae: 0.40, valid loss: 1.00, valid mae: 0.88, test loss: 1.00, test mae: 0.87
Epochs 001, train loss: 0.23, train mae: 0.37, valid loss: 0.19, valid mae: 0.33, test loss: 0.18, test mae: 0.32
INFO:tensorflow:Restoring parameters from tf_sessions/traffic/ltc
Best epoch 001, train loss: 0.267, train mae: 0.404, valid loss: 0.189, valid mae: 0.325, test loss: 0.176, test mae: 0.321


array([[[  0.        ,  -1.4758606 ,   0.        , ...,  12.        ,
           4.        ,   0.86602545],
        [  0.        ,  12.494141  ,   4.05      , ...,  92.        ,
           1.        ,   0.70710677],
        [  0.        ,  10.494141  ,   0.        , ...,   5.        ,
           5.        ,   0.9914449 ],
        ...,
        [  0.        ,  -7.625885  ,   0.        , ...,  90.        ,
           4.        ,   0.49999985],
        [  0.        ,  -7.0558777 ,   0.        , ...,  90.        ,
           1.        ,   0.8660254 ],
        [  0.        , -15.055878  ,   0.        , ...,  90.        ,
           0.        ,   0.70710677]],

       [[  0.        ,  -0.4058838 ,   0.        , ...,  90.        ,
           4.        ,   0.9238795 ],
        [  0.        ,  11.75412   ,   3.8       , ...,  92.        ,
           1.        ,   0.6087615 ],
        [  0.        ,  12.134125  ,   0.        , ...,   5.        ,
           5.        ,   1.        ],
        ...,


In [10]:
model.predict(input_ts_data.test_x)

AttributeError: 'numpy.ndarray' object has no attribute 'iterate_train'

In [18]:
48204/9038

5.333480858597035

In [34]:
type(x)

numpy.ndarray

In [62]:
sequences_x = []
sequences_y = []

for s in range(0, x.shape[0] - seq_len, 4):
    start = s
    end = start + seq_len
    #print("start:", s, "end:", end)
    sequences_x.append(x[start:end])
    sequences_y.append(y[start:end])

x_input_seq = np.stack(sequences_x, axis=1)
y_input_seq = np.stack(sequences_y, axis=1)
total_seqs = x_input_seq.shape[1]

In [65]:
total_seqs

12050

In [68]:
permutation = np.random.RandomState(123).permutation(total_seqs)
valid_size = int(0.1 * total_seqs)
test_size = int(0.15 * total_seqs)

In [69]:
valid_size

1205

In [72]:
valid_x = x_input_seq[:, permutation[:valid_size]]
valid_y = y_input_seq[:, permutation[:valid_size]]
test_x = x_input_seq[:, permutation[valid_size : valid_size + test_size]]
test_y = y_input_seq[:, permutation[valid_size : valid_size + test_size]]
x_input_seq = x_input_seq[:, permutation[valid_size + test_size :]]
y_input_seq = y_input_seq[:, permutation[valid_size + test_size :]]

In [74]:
valid_x.shape, valid_y.shape

((6, 1205, 7), (6, 1205))

In [75]:
test_x.shape, test_y.shape

((6, 1807, 7), (6, 1807))

In [76]:
x_input_seq.shape, y_input_seq.shape

((6, 9038, 7), (6, 9038))

In [193]:
batch_size = 16
total_seqs = x_input_seq.shape[1]
permutation = np.random.permutation(total_seqs)
total_batches = total_seqs // batch_size

for i in range(total_batches):
    start = i * batch_size
    end = start + batch_size
    batch_x = x_input_seq[:, permutation[start:end]]
    batch_y = y_input_seq[:, permutation[start:end]]
    break 

In [194]:
batch_y[0][:1].reshape(1,1)

array([[0.5507145]], dtype=float32)

In [195]:
batch_x[:2].shape

(2, 16, 7)

In [196]:
batch_x.shape

(6, 16, 7)

In [203]:
batch_y[:1]

array([[ 0.5507145 ,  1.2518278 , -1.460016  , -1.0880686 , -0.13227957,
         0.32221097,  0.63778746,  0.51548266,  0.61161524, -0.8847307 ,
        -0.06131261, -0.23797505,  0.5285688 , -0.29434597, -0.91945916,
        -1.4736054 ]], dtype=float32)

In [202]:
batch_x[:1]

array([[[ 0.0000000e+00,  4.3041382e+00,  0.0000000e+00,  0.0000000e+00,
          1.0000000e+00,  2.0000000e+00,  9.6592587e-01],
        [ 0.0000000e+00,  5.6041260e+00,  7.5999999e-01,  0.0000000e+00,
          5.6000000e+01,  0.0000000e+00,  8.6602545e-01],
        [ 0.0000000e+00,  6.8141174e+00,  0.0000000e+00,  0.0000000e+00,
          1.0000000e+00,  3.0000000e+00,  3.8268346e-01],
        [ 0.0000000e+00, -5.1458740e+00,  0.0000000e+00,  0.0000000e+00,
          9.0000000e+01,  6.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  1.5354126e+01,  0.0000000e+00,  0.0000000e+00,
          7.5000000e+01,  2.0000000e+00,  3.8268328e-01],
        [ 0.0000000e+00, -6.8558655e+00,  0.0000000e+00,  0.0000000e+00,
          9.0000000e+01,  4.0000000e+00,  7.9335332e-01],
        [ 0.0000000e+00, -5.3758850e+00,  0.0000000e+00,  0.0000000e+00,
          4.0000000e+01,  6.0000000e+00,  9.6592581e-01],
        [ 0.0000000e+00, -1.9405884e+01,  0.0000000e+00,  0.0000000e+00,
          

In [201]:
_, _, y_hat = model.sess.run(
    [model.accuracy, model.loss, model.y],
    {model.x: batch_x[:1], model.target_y: batch_y[:1]},
)

print("y prediction ", y_hat)

y prediction  [[[ 0.7842461 ]
  [ 1.1884642 ]
  [-0.9494908 ]
  [-1.2282209 ]
  [-0.9567351 ]
  [ 0.975094  ]
  [-0.20665249]
  [ 0.65338534]
  [ 0.669698  ]
  [-1.391655  ]
  [-1.0266671 ]
  [ 0.3377916 ]
  [ 0.23430961]
  [-1.0635269 ]
  [-1.3088336 ]
  [-1.3294183 ]]]


In [199]:
_, _, y_hat = model.sess.run(
    [],
    {model.x: batch_x[:1], model.target_y: batch_y[:1]},
)

print("y prediction ", y_hat)

ValueError: not enough values to unpack (expected 3, got 0)

In [115]:
for batch_x, batch_y in zip(batch_x[0][1].reshape(1,1,7), batch_y[0][:1].reshape(1,1)):
    y_hat = model.sess.run(
        {model.x: batch_x, model.target_y: batch_y},
    )
    if not model.constrain_op is None:
        model.sess.run(model.constrain_op)
    
    print("Input data ", batch_x)
    print("y prediction ", y_hat)

TypeError: Argument `fetch` = [[ 0.0000000e+00  1.5254120e+01  0.0000000e+00  0.0000000e+00
   1.2000000e+01  1.0000000e+00  7.0710677e-01]
 [ 0.0000000e+00  1.0874115e+01  0.0000000e+00  0.0000000e+00
   5.0000000e+00  5.0000000e+00  9.9144489e-01]
 [ 0.0000000e+00 -7.6258850e+00  0.0000000e+00  0.0000000e+00
   9.0000000e+01  4.0000000e+00  6.0876149e-01]
 [ 0.0000000e+00  5.2741394e+00  0.0000000e+00  0.0000000e+00
   1.0000000e+00  2.0000000e+00  2.5881904e-01]
 [ 0.0000000e+00 -2.2858582e+00  0.0000000e+00  0.0000000e+00
   6.4000000e+01  4.0000000e+00  1.3052632e-01]
 [ 0.0000000e+00 -1.1045868e+01  0.0000000e+00  0.0000000e+00
   1.0000000e+00  0.0000000e+00  7.9335338e-01]
 [ 0.0000000e+00 -3.1125870e+01  0.0000000e+00  0.0000000e+00
   1.0000000e+00  2.0000000e+00  5.0000000e-01]
 [ 0.0000000e+00  2.3941345e+00  0.0000000e+00  0.0000000e+00
   2.0000000e+01  6.0000000e+00  7.9335338e-01]
 [ 0.0000000e+00  1.8834137e+01  0.0000000e+00  0.0000000e+00
   7.5000000e+01  6.0000000e